<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/main/Reddit_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setup

In [1]:
!pip install wrds datetime
import pandas as pd, numpy as np, wrds, datetime, sqlite3

# Read datasets

In [2]:
option_list = pd.read_sas("option_ticker.sas7bdat",encoding="utf-8")

# Set search keys

Using UTC time zone and ticker to link reddit posts

## UTC Datetime

In [3]:
option_list['date'] = pd.to_datetime(option_list['date']).dt.tz_localize('US/Eastern')
option_list['date'] = option_list['date'].dt.tz_convert('UTC')

Current search will be based on an interval of 48 hours.

In [4]:
interval = 2
option_list['start_date'] = option_list['date'] - pd.Timedelta(days=interval)

In [5]:
wsb_post = pd.read_csv('r_wallstreetbets_posts.csv',header=0,low_memory=False)

In [6]:
wsb_post.dtypes

id                        object
title                     object
score                      int64
author                    object
author_flair_text         object
removed_by                object
total_awards_received    float64
awarders                  object
created_utc                int64
full_link                 object
num_comments               int64
over_18                     bool
dtype: object

In [7]:
wsb_post['date'] = pd.to_datetime(wsb_post['created_utc'],unit='s')

## Ticker

In [51]:
list(option_list['ticker'])[-1]

'SLDB'

In [49]:
wsb_post['title'].str.findall('|\b'.join(map(str,list(option_list['ticker'].unique()))))

0         []
1         []
2         []
3         []
4         []
          ..
884058    []
884059    []
884060    []
884061    []
884062    []
Name: title, Length: 884063, dtype: object

Variables to capture from posts

* Number of posts
* Combined rewards
* Comments
* Score

# Reddit Scrape

## Web driver

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-driver

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
url = 'https://www.reddit.com/r/wallstreetbets/search/?q=flair%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new'
driver.get(url)

In [ ]:
from datetime import date, timedelta
from dateutil.parser import parse
import re
import praw

In [ ]:
yesterday = date.today() - timedelta(days=1)
links = driver.find_elements_by_xpath('//*[@class="_eYtD2XCVieq6emjKBH3m"]')

r/wallstreetbet was created on January 31, 2012

In [ ]:
pattern = re.compile(r'[ADFJMNOS][a-z]+ \d{1,2},? \d{4}') ## pattern to find date

for a in links:
    if a.text.startswith('Daily Discussion Thread'):
        z = re.search("[ADFJMNOS][a-z]+ \d{1,2},? \d{4}", a.text)
        if z:
            date = z.group()
    parsed = parse(date) 
    if parse(str(yesterday)) == parsed:
        link = a.find_element_by_xpath('../..').get_attribute('href')
       

In [ ]:
 
        ## Not sure if this part is valid. There's a new "Weekend" flair
    if a.text.startswith('Weekend'):
        weekend_date = a.text.split(' ')
        parsed_date = weekend_date[-3] + ' ' + \
            weekend_date[-2].split("-")[1] + weekend_date[-1]
        parsed = parse(parsed_date) 
        saturday = weekend_date[-3] + ' ' + \
            str(int(weekend_date[-2].split("-")[1]
            .replace(',','')) - 1) + ' ' + weekend_date[-1] 

    if parse(str(yesterday)) == parsed: 
        link = a.find_element_by_xpath('../..').get_attribute('href')
    elif parse(str(yesterday)) == parse(str(saturday)):
        link = a.find_element_by_xpath('../..').get_attribute('href')

In [ ]:
### r/wallstreetbet has a flair summary. Check to determine what will be most 
### suitable.

## Reddit API (PRAW)

In [ ]:
reddit = praw.Reddit(
    client_id="JVu4C2dX7-YxNA",
    client_secret="ej0taPaFB--IiWFPYr7KM1kjo8H6UA",
    user_agent="web:test_app:v0.1 (by u/Leo_the_great_1",
    check_for_async=False 
)
wsb = reddit.subreddit("wallstreetbets")

In [ ]:
import pprint
for submission in wsb.hot(limit=1):
    pprint.pprint(vars(submission))

{'_comments_by_id': {},
 '_fetched': False,
 '_reddit': <praw.reddit.Reddit object at 0x7f47bace0b10>,
 'all_awardings': [{'award_sub_type': 'PREMIUM',
                    'award_type': 'global',
                    'awardings_required_to_grant_benefits': None,
                    'coin_price': 30,
                    'coin_reward': 0,
                    'count': 2,
                    'days_of_drip_extension': 0,
                    'days_of_premium': 0,
                    'description': 'A glowing commendation for all to see',
                    'end_date': None,
                    'giver_coin_reward': None,
                    'icon_format': 'APNG',
                    'icon_height': 2048,
                    'icon_url': 'https://www.redditstatic.com/gold/awards/icon/Illuminati_512.png',
                    'icon_width': 2048,
                    'id': 'award_b4ff447e-05a5-42dc-9002-63568807cfe6',
                    'is_enabled': True,
                    'is_new': False,
     

In [ ]:
#### Can we construct an index of retail investor "frenzy"?
